In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
import pandas as pd
import numpy as np

In [2]:
app = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')
previous_app = pd.read_csv('data/previous_application.csv')
install = pd.read_csv('data/installments_payments.csv')

In [3]:
print(f"app shape {app.shape}")
print(f"previous app shape {previous_app.shape}")
print(f"installments shape {install.shape}")

app shape (307511, 122)
previous app shape (1670214, 37)
installments shape (13605401, 8)


## PROCESS INSTALLMENTS

In [13]:
install.head(5)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


Cell that customers did not repay


In [12]:
install.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

DIF_DAYS_INSTALMENT: differences in days applicant paid back, positive values meaning late repay (lower is better, positive is bad)

DIF_AMT_PAYMENT: How much remaining money user need to pay (lower is worst, 0 is best)

In [4]:
install.DAYS_ENTRY_PAYMENT = abs(install.DAYS_ENTRY_PAYMENT)
install.DAYS_INSTALMENT = abs(install.DAYS_INSTALMENT)
install['DIF_DAYS_INSTALMENT'] = install.DAYS_INSTALMENT - install.DAYS_ENTRY_PAYMENT
install['DIF_AMT_PAYMENT'] = install.AMT_PAYMENT - install.AMT_INSTALMENT

In [79]:
install.head(5)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DIF_DAYS_INSTALMENT,DIF_AMT_PAYMENT
0,1054186,161674,1.0,6,1180.0,1187.0,6948.360,6948.360,-7.0,0.000
1,1330831,151639,0.0,34,2156.0,2156.0,1716.525,1716.525,0.0,0.000
2,2085231,193053,2.0,1,63.0,63.0,25425.000,25425.000,0.0,0.000
3,2452527,199697,1.0,3,2418.0,2426.0,24350.130,24350.130,-8.0,0.000
4,2714724,167756,1.0,2,1383.0,1366.0,2165.040,2160.585,17.0,-4.455


In [5]:
group_install = install.groupby('SK_ID_CURR', as_index=False).count().iloc[:,:2]
group_install.columns = ['SK_ID_CURR', 'NUM_INSTALMENT']

In [6]:
install_na = install[install.isnull().any(axis=1)]
group_install_na = install_na.groupby(['SK_ID_CURR'], as_index=False).count().iloc[:,:2]
group_install_na.columns = ['SK_ID_CURR', 'NUM_NOT_INSTALMENT']

In [7]:
group_install = group_install.merge(group_install_na, on='SK_ID_CURR', how='left')
group_install['NUM_NOT_INSTALMENT'] = group_install['NUM_NOT_INSTALMENT'].fillna(0)

In [8]:
temp1 = install.groupby('SK_ID_CURR', as_index=False)['AMT_INSTALMENT'].sum()
temp2 = install.groupby('SK_ID_CURR', as_index=False)['AMT_PAYMENT'].sum()
temp1 = temp1.merge(temp2, on='SK_ID_CURR', how='left')
temp1['DIF_AMT_PAYMENT'] = temp1['AMT_PAYMENT'] - temp1['AMT_INSTALMENT']
temp1 = temp1.drop(['AMT_INSTALMENT', 'AMT_PAYMENT'], axis=1)

group_install = group_install.merge(temp1, on='SK_ID_CURR', how='left')

In [9]:
temp1 = install.groupby('SK_ID_CURR', as_index=False)['DIF_DAYS_INSTALMENT'].mean()
temp2 = install.groupby('SK_ID_CURR', as_index=False)['DIF_DAYS_INSTALMENT'].min()
temp3 = install.groupby('SK_ID_CURR', as_index=False)['DIF_DAYS_INSTALMENT'].max()
temp4 = install.groupby('SK_ID_CURR', as_index=False)['DIF_DAYS_INSTALMENT'].sum()

In [10]:
idx_null = temp1[temp1['DIF_DAYS_INSTALMENT'].isnull()].index.tolist()
for idx in idx_null:
    temp1['DIF_DAYS_INSTALMENT'].iloc[idx] = install[install.SK_ID_CURR == temp1['SK_ID_CURR'].iloc[idx]]['DAYS_INSTALMENT'].tolist()[0]
    temp2['DIF_DAYS_INSTALMENT'].iloc[idx] = install[install.SK_ID_CURR == temp1['SK_ID_CURR'].iloc[idx]]['DAYS_INSTALMENT'].tolist()[0]
    temp3['DIF_DAYS_INSTALMENT'].iloc[idx] = install[install.SK_ID_CURR == temp1['SK_ID_CURR'].iloc[idx]]['DAYS_INSTALMENT'].tolist()[0]
    temp4['DIF_DAYS_INSTALMENT'].iloc[idx] = install[install.SK_ID_CURR == temp1['SK_ID_CURR'].iloc[idx]]['DAYS_INSTALMENT'].tolist()[0]

temp1.columns = ['SK_ID_CURR', 'DIF_DAYS_INSTALMENT_mean']
temp2.columns = ['SK_ID_CURR', 'DIF_DAYS_INSTALMENT_min']
temp3.columns = ['SK_ID_CURR', 'DIF_DAYS_INSTALMENT_max']
temp4.columns = ['SK_ID_CURR', 'DIF_DAYS_INSTALMENT_sum']



In [11]:
group_install = group_install.merge(temp1, on='SK_ID_CURR', how='left')
group_install = group_install.merge(temp2, on='SK_ID_CURR', how='left')
group_install = group_install.merge(temp3, on='SK_ID_CURR', how='left')
group_install = group_install.merge(temp4, on='SK_ID_CURR', how='left')

In [269]:
group_install.head()

,SK_ID_CURR,NUM_INSTALMENT,NUM_NOT_INSTALMENT,DIF_AMT_PAYMENT,DIF_DAYS_INSTALMENT_mean,DIF_DAYS_INSTALMENT_min,DIF_DAYS_INSTALMENT_max,DIF_DAYS_INSTALMENT_sum
0,100001,7,0.0,0.0,-7.285714,-36.0,11.0,-51.0
1,100002,19,0.0,0.0,-20.421053,-31.0,-12.0,-388.0
2,100003,25,0.0,0.0,-7.160000,-14.0,-1.0,-179.0
3,100004,3,0.0,0.0,-7.666667,-11.0,-3.0,-23.0
4,100005,9,0.0,0.0,-23.555556,-37.0,1.0,-212.0


In [12]:
group_install['PORTION_NOT_INSTALMENT'] = group_install['NUM_NOT_INSTALMENT'] / group_install['NUM_INSTALMENT']
group_install['AVG_DIF_AMT_PAYMENT'] = 0
for idx in range(len(group_install)):
    if group_install['NUM_INSTALMENT'][idx] == group_install['NUM_NOT_INSTALMENT'][idx]:
        group_install['AVG_DIF_AMT_PAYMENT'][idx] = group_install['DIF_AMT_PAYMENT'][idx]
    else:   
        group_install['AVG_DIF_AMT_PAYMENT'][idx] = group_install['DIF_AMT_PAYMENT'][idx] / (group_install['NUM_INSTALMENT'][idx] - group_install['NUM_NOT_INSTALMENT'][idx])
group_install = group_install.drop(['NUM_INSTALMENT', 'NUM_NOT_INSTALMENT', 'DIF_AMT_PAYMENT'], axis=1)
group_install.head(10)

,SK_ID_CURR,DIF_DAYS_INSTALMENT_mean,DIF_DAYS_INSTALMENT_min,DIF_DAYS_INSTALMENT_max,DIF_DAYS_INSTALMENT_sum,PORTION_NOT_INSTALMENT,AVG_DIF_AMT_PAYMENT
0,100001,-7.285714,-36.0,11.0,-51.0,0.0,0
1,100002,-20.421053,-31.0,-12.0,-388.0,0.0,0
2,100003,-7.160000,-14.0,-1.0,-179.0,0.0,0
3,100004,-7.666667,-11.0,-3.0,-23.0,0.0,0
4,100005,-23.555556,-37.0,1.0,-212.0,0.0,0
5,100006,-19.375000,-77.0,-1.0,-310.0,0.0,0
6,100007,-3.636364,-31.0,12.0,-240.0,0.0,-452
7,100008,26.114286,-28.0,1317.0,914.0,0.0,-342
8,100009,-8.588235,-23.0,7.0,-438.0,0.0,0
9,100010,-11.900000,-30.0,-5.0,-119.0,0.0,0


In [271]:
np.all(np.isfinite(group_install))

True

## PROCESS PREVIOUS APP

In [192]:
previous_app.head(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0
6,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
7,1656711,296299,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,7,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
8,2367563,342292,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
9,2579447,334349,Cash loans,NaN,0.0,0.0,NaN,NaN,SATURDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
missing_dict = dict(previous_app.isnull().sum())
drop_ft = []
for k, v in missing_dict.items():
    if v > 100:
        drop_ft += [k]
len(drop_ft)

15

In [243]:
missing_dict

{'SK_ID_PREV': 0,
 'SK_ID_CURR': 0,
 'NAME_CONTRACT_TYPE': 0,
 'AMT_ANNUITY': 372235,
 'AMT_APPLICATION': 0,
 'AMT_CREDIT': 1,
 'AMT_DOWN_PAYMENT': 895844,
 'AMT_GOODS_PRICE': 385515,
 'WEEKDAY_APPR_PROCESS_START': 0,
 'HOUR_APPR_PROCESS_START': 0,
 'FLAG_LAST_APPL_PER_CONTRACT': 0,
 'NFLAG_LAST_APPL_IN_DAY': 0,
 'RATE_DOWN_PAYMENT': 895844,
 'RATE_INTEREST_PRIMARY': 1664263,
 'RATE_INTEREST_PRIVILEGED': 1664263,
 'NAME_CASH_LOAN_PURPOSE': 0,
 'NAME_CONTRACT_STATUS': 0,
 'DAYS_DECISION': 0,
 'NAME_PAYMENT_TYPE': 0,
 'CODE_REJECT_REASON': 0,
 'NAME_TYPE_SUITE': 820405,
 'NAME_CLIENT_TYPE': 0,
 'NAME_GOODS_CATEGORY': 0,
 'NAME_PORTFOLIO': 0,
 'NAME_PRODUCT_TYPE': 0,
 'CHANNEL_TYPE': 0,
 'SELLERPLACE_AREA': 0,
 'NAME_SELLER_INDUSTRY': 0,
 'CNT_PAYMENT': 372230,
 'NAME_YIELD_GROUP': 0,
 'PRODUCT_COMBINATION': 346,
 'DAYS_FIRST_DRAWING': 673065,
 'DAYS_FIRST_DUE': 673065,
 'DAYS_LAST_DUE_1ST_VERSION': 673065,
 'DAYS_LAST_DUE': 673065,
 'DAYS_TERMINATION': 673065,
 'NFLAG_INSURED_ON_APPROVAL

In [14]:
previous_drop = previous_app.drop(drop_ft, axis=1)
previous_drop = previous_drop.dropna()

In [246]:
previous_drop.isnull().sum()

SK_ID_PREV                     0
SK_ID_CURR                     0
NAME_CONTRACT_TYPE             0
AMT_APPLICATION                0
AMT_CREDIT                     0
WEEKDAY_APPR_PROCESS_START     0
HOUR_APPR_PROCESS_START        0
FLAG_LAST_APPL_PER_CONTRACT    0
NFLAG_LAST_APPL_IN_DAY         0
NAME_CASH_LOAN_PURPOSE         0
NAME_CONTRACT_STATUS           0
DAYS_DECISION                  0
NAME_PAYMENT_TYPE              0
CODE_REJECT_REASON             0
NAME_CLIENT_TYPE               0
NAME_GOODS_CATEGORY            0
NAME_PORTFOLIO                 0
NAME_PRODUCT_TYPE              0
CHANNEL_TYPE                   0
SELLERPLACE_AREA               0
NAME_SELLER_INDUSTRY           0
NAME_YIELD_GROUP               0
dtype: int64

In [227]:
unique_id = previous_drop.SK_ID_CURR.unique()

In [228]:
len(unique_id)

338857

In [15]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [16]:
group_previous = count_categorical(previous_drop[['SK_ID_CURR', 'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE']], 'SK_ID_CURR', 'previous')

In [17]:
previous_drop['DIF_AMT_CREDIT'] = previous_drop.AMT_CREDIT - previous_drop.AMT_APPLICATION

In [251]:
previous_drop.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_APPLICATION,AMT_CREDIT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,...,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,DIF_AMT_CREDIT
0,2030495,271877,Consumer loans,17145.0,17145.0,SATURDAY,15,Y,1,XAP,...,XAP,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,middle,0.0
1,2802425,108129,Cash loans,607500.0,679671.0,THURSDAY,11,Y,1,XNA,...,XAP,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,low_action,72171.0
2,2523466,122040,Cash loans,112500.0,136444.5,TUESDAY,11,Y,1,XNA,...,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,high,23944.5
3,2819243,176158,Cash loans,450000.0,470790.0,MONDAY,7,Y,1,XNA,...,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,middle,20790.0
4,1784265,202054,Cash loans,337500.0,404055.0,THURSDAY,9,Y,1,Repairs,...,HC,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,high,66555.0


In [18]:
temp1 = previous_drop.groupby('SK_ID_CURR', as_index=False)['DIF_AMT_CREDIT'].mean()
temp2 = previous_drop.groupby('SK_ID_CURR', as_index=False)['DIF_AMT_CREDIT'].min()
temp3 = previous_drop.groupby('SK_ID_CURR', as_index=False)['DIF_AMT_CREDIT'].max()
temp4 = previous_drop.groupby('SK_ID_CURR', as_index=False)['DIF_AMT_CREDIT'].sum()

temp1.columns = ['SK_ID_CURR', 'DIF_AMT_CREDIT_mean']
temp2.columns = ['SK_ID_CURR', 'DIF_AMT_CREDIT_min']
temp3.columns = ['SK_ID_CURR', 'DIF_AMT_CREDIT_max']
temp4.columns = ['SK_ID_CURR', 'DIF_AMT_CREDIT_sum']

In [19]:
group_previous = group_previous.merge(temp1, on='SK_ID_CURR', how='left')
group_previous = group_previous.merge(temp2, on='SK_ID_CURR', how='left')
group_previous = group_previous.merge(temp3, on='SK_ID_CURR', how='left')
group_previous = group_previous.merge(temp4, on='SK_ID_CURR', how='left')

In [20]:
group_previous.shape

(338857, 21)

## CONCATE WITH APPLICATION 

In [21]:
concate_data = app.merge(group_install, on='SK_ID_CURR', how='left')
concate_data = concate_data.merge(group_previous, on='SK_ID_CURR', how='left')

In [273]:
concate_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,previous_NAME_CONTRACT_TYPE_Consumer loans_count,previous_NAME_CONTRACT_TYPE_Consumer loans_count_norm,previous_NAME_CONTRACT_TYPE_Revolving loans_count,previous_NAME_CONTRACT_TYPE_Revolving loans_count_norm,previous_NAME_CONTRACT_TYPE_XNA_count,previous_NAME_CONTRACT_TYPE_XNA_count_norm,DIF_AMT_CREDIT_mean,DIF_AMT_CREDIT_min,DIF_AMT_CREDIT_max,DIF_AMT_CREDIT_sum
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.0,1.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,2.0,0.666667,0.0,0.000000,0.0,0.0,48754.50,-756.0,135882.0,146263.50
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.0,1.000000,0.0,0.000000,0.0,0.0,-4176.00,-4176.0,-4176.0,-4176.00
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2.0,0.222222,2.0,0.222222,0.0,0.0,19492.24,-66987.0,218115.0,175430.16
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,2.0,0.333333,0.0,0.000000,0.0,0.0,16108.50,-2560.5,59400.0,96651.00


In [274]:
concate_data.to_csv('data/merged_application_train.csv', index=False)

In [276]:
concate_data = test.merge(group_install, on='SK_ID_CURR', how='left')
concate_data = concate_data.merge(group_previous, on='SK_ID_CURR', how='left')

In [277]:
concate_data.to_csv('data/merged_application_test.csv', index=False)